In [2]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

In [3]:
student = pd.read_csv('/Users/newuser/Desktop/saas/DF Project/student-lpor.csv')


In [4]:
student.head(10)

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,no,4,3,4,1,1,3,4,0,11
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,no,5,3,3,1,1,3,2,9,11
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,no,4,3,2,2,3,3,6,12,13
3,GP,F,15,U,GT3,T,4,2,health,services,...,yes,3,2,2,1,1,5,0,14,14
4,GP,F,16,U,GT3,T,3,3,other,other,...,no,4,3,2,1,2,5,0,11,13
5,GP,M,16,U,LE3,T,4,3,services,other,...,no,5,4,2,1,2,5,6,12,12
6,GP,M,16,U,LE3,T,2,2,other,other,...,no,4,4,4,1,1,3,0,13,12
7,GP,F,17,U,GT3,A,4,4,other,teacher,...,no,4,1,4,1,1,1,2,10,13
8,GP,M,15,U,LE3,A,3,2,services,other,...,no,4,2,2,1,1,1,0,15,16
9,GP,M,15,U,GT3,T,3,4,other,other,...,no,5,5,1,1,1,5,0,12,12
